## 特征工程

In [5]:
# 工具导入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import gc
from collections import Counter
import copy

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [6]:
# 数据读取
test_data = pd.read_csv('dataset/data_format1/test_format1.csv')
train_data = pd.read_csv('dataset/data_format1/train_format1.csv')

user_info = pd.read_csv('dataset/data_format1/user_info_format1.csv')
user_log = pd.read_csv('dataset/data_format1/user_log_format1.csv')

In [7]:
# 对数据进行内存压缩
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    numerics = ['int16', 'int32','int64','float16','float32','float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('decrease by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
num_rows = None
train_file = './dataset/data_format1/train_format1.csv'
test_file = './dataset/data_format1/test_format1.csv'

user_info_file = './dataset/data_format1/user_info_format1.csv'
user_log_file = './dataset/data_format1/user_log_format1.csv'

# train_data = reduce_mem_usage(pd.read_csv(train_file, num_rows))
# test_data = reduce_mem_usage(pd.read_csv(test_file, num_rows))
# user_info = reduce_mem_usage(pd.read_csv(user_info_file, num_rows))
# user_log = reduce_mem_usage(pd.read_csv(user_log_file, num_rows)) # 运行太慢

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB


In [10]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261477 entries, 0 to 261476
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      261477 non-null  int64  
 1   merchant_id  261477 non-null  int64  
 2   prob         0 non-null       float64
dtypes: float64(1), int64(2)
memory usage: 6.0 MB


In [11]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  421953 non-null  float64
 2   gender     417734 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [12]:
user_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   seller_id    int64  
 4   brand_id     float64
 5   time_stamp   int64  
 6   action_type  int64  
dtypes: float64(1), int64(6)
memory usage: 2.9 GB


In [13]:
# 数据处理
all_data = train_data.append(test_data)
all_data = all_data.merge(user_info, on=['user_id'], how='left')
del train_data, test_data, user_info
gc.collect()

61

In [14]:
all_data.head()

,user_id,merchant_id,label,prob,age_range,gender
0,34176,3906,0.0,NaN,6.0,0.0
1,34176,121,0.0,NaN,6.0,0.0
2,34176,4356,1.0,NaN,6.0,0.0
3,34176,2217,0.0,NaN,6.0,0.0
4,230784,4818,0.0,NaN,0.0,0.0


In [15]:
# 按时间排序
user_log = user_log.sort_values(['user_id', 'time_stamp'])

In [16]:
# 对每个用户合并所有字段，合并字段为item_id
list_join_func = lambda x: " ".join([str(i) for i in x])

agg_dict = {
    'item_id': list_join_func,
    'cat_id': list_join_func,
    'seller_id': list_join_func,
    'brand_id': list_join_func,
    'time_stamp': list_join_func,
    'action_type': list_join_func,
}

rename_dict = {
    'item_id': 'item_path',
    'cat_id': 'cat_path',
    'seller_id': 'seller_path',
    'brand_id': 'brand_path',
    'time_stamp': 'time_stamp_path',
    'action_type': 'action_type_path',
}

def merge_list(df_ID, join_columns, df_data, agg_dict, rename_dict):
    df_data = df_data\
        .groupby(join_columns)\
        .agg(agg_dict)\
        .reset_index()\
        .rename(columns=rename_dict)
    df_ID = df_ID.merge(df_data, on=join_columns, how='left')
    return df_ID

all_data = merge_list(all_data, 'user_id', user_log, agg_dict, rename_dict)

In [17]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522341 entries, 0 to 522340
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           522341 non-null  int64  
 1   merchant_id       522341 non-null  int64  
 2   label             260864 non-null  float64
 3   prob              0 non-null       float64
 4   age_range         519763 non-null  float64
 5   gender            514796 non-null  float64
 6   item_path         522341 non-null  object 
 7   cat_path          522341 non-null  object 
 8   seller_path       522341 non-null  object 
 9   brand_path        522341 non-null  object 
 10  time_stamp_path   522341 non-null  object 
 11  action_type_path  522341 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 51.8+ MB


In [18]:
all_data

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,601 601 601 601 614 614 614 614 614 614 618 61...,0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,NaN,NaN,6.0,0.0,802791 977305 351177 122937 21972 863063 10903...,602 602 602 602 552 1271 1271 662 662 821 662 ...,2823 2823 2664 2664 1076 2946 2781 4949 2412 4...,1128.0 1128.0 8154.0 8154.0 3549.0 5559.0 3305...,511 511 512 512 512 516 516 521 521 521 521 52...,3 3 2 2 2 3 3 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 ...
522337,97919,2341,NaN,NaN,8.0,1.0,484765 128769 128769 995386 128769 645625 9953...,737 464 464 464 464 464 464 464 464 464 464 46...,4408 235 235 235 235 3416 235 235 235 235 235 ...,6967.0 2020.0 2020.0 2020.0 2020.0 6242.0 2020...,626 707 707 710 710 710 710 710 710 710 710 71...,2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 3 ...
522338,97919,3971,NaN,NaN,8.0,1.0,484765 128769 128769 995386 128769 645625 9953...,737 464 464 464 464 464 464 464 464 464 464 46...,4408 235 235 235 235 3416 235 235 235 235 235 ...,6967.0 2020.0 2020.0 2020.0 2020.0 6242.0 2020...,626 707 707 710 710 710 710 710 710 710 710 71...,2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 3 ...
522339,32639,3536,NaN,NaN,0.0,0.0,394570 394570 394570 28017 110194 314126 95836...,1413 1413 1413 812 1271 1271 1271 1198 1271 11...,1065 1065 1065 1506 38 1890 2280 4873 2280 487...,6374.0 4468.0 6374.0 4888.0 7010.0 5683.0 5372...,523 523 523 525 617 617 723 723 723 723 807 81...,0 2 0 0 0 0 0 0 0 2 0 0 0 0 2 0 2 0 0 3 0 0 0 ...


In [19]:
del user_log
gc.collect()

19

In [20]:
# 定义特征统计函数
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

def nunique(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1

def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        return -1

def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        return -1

def std_(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        return -1
# 定义统计数据中对topN数据的函数
def most_n(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        return -1
# 定义统计数据中对topN数据总数的函数
def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        return -1

In [21]:
# 调用定义的统计函数
'''
df_data: 数据
single_col：要进行统计的列名
name：统计完成新建的列名
'''
def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data

def user_nunique(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(nunique)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data

def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

### 提取统计特征

In [22]:
# 特征统计
# 店铺特征统计：统计与店铺特点有关的特征，如店铺、商品、品牌等
all_data_test = all_data.head(2000)
# 统计用户点击、浏览、加购、购买行为
# 总次数
all_data_test = user_cnt(all_data_test, 'seller_path', 'user_cnt')
# 不同店铺个数
all_data_test = user_nunique(all_data_test, 'seller_path', 'seller_nunique')
# 不同品类个数
all_data_test = user_nunique(all_data_test, 'cat_path', 'cat_nunique')
# 不同品牌个数
all_data_test = user_nunique(all_data_test, 'brand_path', 'brand_nunique')
# 不同商品个数
all_data_test = user_nunique(all_data_test, 'item_path', 'item_nunique')
# 活跃天数
all_data_test = user_nunique(all_data_test, 'time_stamp_path', 'time_stamp_nunique')
# 不同用户行为种数
all_data_test = user_nunique(all_data_test, 'action_type_path', 'action_type_nunique')
# 最晚时间
all_data_test = user_max(all_data_test, 'action_type_path', 'time_stamp_max')
# 最早时间
all_data_test = user_min(all_data_test, 'action_type_path', 'time_stamp_min')
# 活跃天数方差
all_data_test = user_std(all_data_test, 'action_type_path', 'time_stamp_std')
# 最早和最晚相差天数
all_data_test['time_stamp_range'] = all_data_test['time_stamp_max'] - all_data_test['time_stamp_min']
# 用户最喜欢的店铺
all_data_test = user_most_n(all_data_test, 'seller_path', 'seller_most_1', n=1)
# 最喜欢的类目
all_data_test = user_most_n(all_data_test, 'cat_path', 'cat_most_1', n=1)
# 最喜欢的品牌
all_data_test = user_most_n(all_data_test, 'brand_path', 'brand_most_1', n=1)
# 最常见的行为动作
all_data_test = user_most_n(all_data_test, 'action_type_path', 'action_type_most_1', n=1)
# ...
# 用户最喜欢的店铺 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'seller_path', 'seller_most_1_cnt', n=1)
# 最喜欢的类目 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'cat_path', 'cat_most_1_cnt', n=1)
# 最喜欢的品牌 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'brand_path', 'brand_most_1_cnt', n=1)
# 最常见的行为动作 行为次数
all_data_test = user_most_n_cnt(all_data_test, 'action_type_path', 'action_type_most_1_cnt', n=1)

In [23]:
all_data_test.describe()

,user_id,merchant_id,label,prob,age_range,gender,user_cnt,seller_nunique,cat_nunique,brand_nunique,...,time_stamp_nunique,action_type_nunique,time_stamp_max,time_stamp_min,time_stamp_std,time_stamp_range,seller_most_1_cnt,cat_most_1_cnt,brand_most_1_cnt,action_type_most_1_cnt
count,2000.000000,2000.000000,2000.000000,0.0,1995.000000,1983.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000,2000.000000
mean,208239.475500,2533.319000,0.064500,NaN,3.135840,0.349975,140.082500,35.701500,23.987000,35.374000,...,17.845500,2.63700,2.577000,0.004000,0.743902,2.57300,29.575000,30.53200,29.844500,123.778000
std,122331.596829,1457.359826,0.245703,NaN,1.889127,0.530178,202.303623,35.853021,18.091365,35.298146,...,14.733505,0.54807,0.494159,0.089376,0.264871,0.50279,46.845711,39.37676,47.037364,186.462639
min,132.000000,10.000000,0.000000,NaN,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,...,2.000000,1.00000,2.000000,0.000000,0.000000,0.00000,1.000000,1.00000,1.000000,2.000000
25%,100930.500000,1329.500000,0.000000,NaN,2.000000,0.000000,39.000000,14.000000,12.000000,14.000000,...,7.000000,2.00000,2.000000,0.000000,0.554763,2.00000,9.000000,10.00000,9.000000,32.000000
50%,206466.000000,2455.500000,0.000000,NaN,3.000000,0.000000,84.000000,25.000000,20.000000,25.000000,...,13.000000,3.00000,3.000000,0.000000,0.706387,3.00000,16.000000,19.00000,16.000000,70.000000
75%,313024.500000,3910.000000,0.000000,NaN,4.000000,1.000000,163.000000,46.000000,31.000000,45.000000,...,24.000000,3.00000,3.000000,0.000000,0.903796,3.00000,32.250000,34.00000,32.000000,141.250000
max,424068.000000,4993.000000,1.000000,NaN,8.000000,2.000000,2470.000000,344.000000,161.000000,320.000000,...,105.000000,4.00000,3.000000,2.000000,1.475535,3.00000,966.000000,532.00000,966.000000,2270.000000


In [24]:
# 用户特征统计：对用户对点击、加购、购买、收藏等特征进行统计。
def col_cnt_(df_data, columns_list, action_type):
    try:
        data_dict = {}
        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']
        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
        path_len = len(data_dict[col])

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            data_out.append(data_txt)
        return len(data_out)
    except:
        return -1

def col_nunique_(df_data, columns_list, action_type):
    try:
        data_dict = {}
        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']
        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
        path_len = len(data_dict[col])

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            data_out.append(data_txt)
        return len(set(data_out))
    except:
        return -1

'''
columns_list: 要计算的列名
action_type：
'''
def user_col_cnt(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_cnt_(x, columns_list, action_type), axis=1)
    return df_data

def user_col_nunique(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_nunique_(x, columns_list, action_type), axis=1)
    return df_data

In [25]:
# 统计用户和店铺的关系：对店铺的用户点击次数、加购次数、购买次数、收藏次数等
# 点击次数
all_data_test = user_col_cnt(all_data_test, ['seller_path'], '0', 'user_cnt_0')
# 加购次数
all_data_test = user_col_cnt(all_data_test, ['seller_path'], '1', 'user_cnt_1')

In [26]:
all_data_test

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,...,seller_most_1,cat_most_1,brand_most_1,action_type_most_1,seller_most_1_cnt,cat_most_1_cnt,brand_most_1_cnt,action_type_most_1_cnt,user_cnt_0,user_cnt_1
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,331,662,4094.0,0,70,98,70,410,451,451
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,331,662,4094.0,0,70,98,70,410,451,451
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,331,662,4094.0,0,70,98,70,410,451,451
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,...,331,662,4094.0,0,70,98,70,410,451,451
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,...,3556,407,1236.0,0,10,9,10,47,54,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,220293,2883,0.0,NaN,4.0,1.0,551995 497734 703172 468886 813265 1424 1424 1...,50 50 683 683 331 1401 1401 1401 1401 1401 737...,638 2782 2884 192 2982 2982 2982 2982 2982 298...,1444.0 84.0 7239.0 5638.0 5750.0 5750.0 5750.0...,...,2677,656,1579.0,0,17,14,17,41,45,45
1996,155013,3727,0.0,NaN,2.0,0.0,881928 833279 961513 305410 305410 305410 1057...,737 737 737 1349 1349 1349 737 737 737 737 737...,4390 3718 4993 702 702 702 3727 3727 3727 3727...,5223.0 3169.0 1865.0 1360.0 1360.0 1360.0 6816...,...,3727,737,6816.0,0,47,51,47,104,106,106
1997,24453,1487,0.0,NaN,5.0,1.0,1049364 1049364 282508 251929 291529 166817 45...,1075 1075 1213 1438 1200 407 1611 407 407 178 ...,3365 3365 1256 639 4016 4346 4346 4346 4346 43...,7524.0 7524.0 1573.0 1565.0 4358.0 2919.0 2919...,...,4346,407,2919.0,0,42,25,42,69,81,81
1998,155781,1861,0.0,NaN,0.0,2.0,437216 1040600 305379 168779 463220 895836 305...,407 407 1438 1438 178 1505 656 612 1553 184 18...,3163 118 4211 4685 4645 3910 816 4364 1910 203...,3881.0 6020.0 5242.0 7647.0 7837.0 5484.0 5784...,...,1199,1389,5409.0,0,5,32,5,55,59,59
